In [1]:
import findspark
findspark.init('/home/asif/spark-2.1.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dfAndCleaning').getOrCreate()

## <span style="color:red">If "findspark" produces import error run *"pip install findspark"* in terminal</span>

# <span style="color:blue">Reading and manipulating Dataset as Spark Dataframe</span>

In [2]:
# read a csv dataset
csvDataFrame = spark.read.csv('ContainsNull.csv', inferSchema = True, header = True)
csvDataFrame.show()

+--------+---------+--------+------------+-----------+-----+
|clientid|     date|weekdays| gains value|     prices|   up|
+--------+---------+--------+------------+-----------+-----+
|       0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
|       1| 5/1/2008|     Thu|        null|3167180.737| true|
|       2| 5/2/2008|     Fri|-0.868509701|9589766.961|false|
|       3| 5/3/2008|     Sat|-0.427010839|       null|false|
|       4| 5/4/2008|    null| 0.253255365|937163.4438| true|
|       5| 5/5/2008|     Mon|-0.681516369|949579.8802|false|
|       6| 5/6/2008|     Tue| 0.007191158|7268426.907| null|
|       7| 5/7/2008|     Wed| 0.674497472|7517014.783| true|
|       8| 5/8/2008|     Thu|-1.184100866|1920959.542|false|
|       9| 5/9/2008|    null| -1.58036926| 8456240.62| null|
|       0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
+--------+---------+--------+------------+-----------+-----+



In [3]:
# read a json dataset
jsonDataFrame = spark.read.json('people.json')
jsonDataFrame.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [4]:
# name of the columns as a list
jsonDataColumns = jsonDataFrame.columns
csvDataFrameColumns = csvDataFrame.columns

print("csvDataFrameColumns list = {0}\njsonDataColumns list = {1}".format(csvDataFrameColumns, jsonDataColumns))

csvDataFrameColumns list = ['clientid', 'date', 'weekdays', 'gains value', 'prices', 'up']
jsonDataColumns list = ['age', 'name']


In [5]:
# Print the schema in a tree format
csvDataFrame.printSchema()

root
 |-- clientid: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- weekdays: string (nullable = true)
 |-- gains value: double (nullable = true)
 |-- prices: double (nullable = true)
 |-- up: boolean (nullable = true)



In [6]:
jsonDataFrame.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



## <span style="color:red">*Sometimes Spark can't infer the exact schema, in that case we have to change the schema programatically.*</span> 
### [Programmatically Specifying the Schema](https://spark.apache.org/docs/2.1.0/sql-programming-guide.html#programmatically-specifying-the-schema)

In [7]:
# jsonDataFrame.printSchema()  describes 'age' column as long, we want it as Integer type
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
data_schema = [StructField('age',IntegerType(),True),
               StructField('name',StringType(),True)]
# true is used to except null values
final_struct = StructType(fields = data_schema)
jsonNewDataFrame = spark.read.json('people.json', schema=final_struct)
jsonNewDataFrame.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [8]:
jsonNewDataFrame.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## *We can also change datatypes by explicit type casting*

In [9]:
# now we change the data type of "clientid" from integer to Double though its not necessary 
from pyspark.sql.types import DoubleType

csvNewDataFrame = csvDataFrame.withColumn("clientid", csvDataFrame["clientid"].cast(DoubleType()))
csvNewDataFrame.printSchema()

root
 |-- clientid: double (nullable = true)
 |-- date: string (nullable = true)
 |-- weekdays: string (nullable = true)
 |-- gains value: double (nullable = true)
 |-- prices: double (nullable = true)
 |-- up: boolean (nullable = true)



## Renaming column

In [10]:
# rename 'clientid' to 'client_id'
csvNewDataFrame = csvNewDataFrame.withColumnRenamed("clientid", "client_id")
csvNewDataFrame.printSchema()

root
 |-- client_id: double (nullable = true)
 |-- date: string (nullable = true)
 |-- weekdays: string (nullable = true)
 |-- gains value: double (nullable = true)
 |-- prices: double (nullable = true)
 |-- up: boolean (nullable = true)



## *change space seperated column name to '_' separeted*

In [11]:

import re
csvNewDataFrame = csvNewDataFrame.toDF(*(re.sub(r'[\.\s]+', '_', c) for c in csvNewDataFrame.columns))
csvNewDataFrame.columns

['client_id', 'date', 'weekdays', 'gains_value', 'prices', 'up']

In [12]:
# print some rows, this head() function returns a list
csvNewDataFrame.head(2)

[Row(client_id=0.0, date='4/30/2008', weekdays='Wed', gains_value=-0.524581929, prices=7791404.009, up=False),
 Row(client_id=1.0, date='5/1/2008', weekdays='Thu', gains_value=None, prices=3167180.737, up=True)]

In [13]:
# print number of rows
numRowCsv = csvNewDataFrame.count()
numRowJson = jsonNewDataFrame.count()
print("numRowCsv = {0} numRowJson = {1}".format(numRowCsv, numRowJson))

numRowCsv = 11 numRowJson = 3


## <span style="color:blue"> *Finding some useful stats about data* </span>

In [14]:
# we can find some statistics using "describe().show()"
csvNewDataFrame.describe().show()

+-------+------------------+---------+--------+------------------+------------------+
|summary|         client_id|     date|weekdays|       gains_value|            prices|
+-------+------------------+---------+--------+------------------+------------------+
|  count|                11|       11|       9|                10|                10|
|   mean| 4.090909090909091|     null|    null|     -0.4855726898| 5538914.089199999|
| stddev|3.1766191290283903|     null|    null|0.6677054107117851|3381724.3034271337|
|    min|               0.0|4/30/2008|     Fri|       -1.58036926|       937163.4438|
|    max|               9.0| 5/9/2008|     Wed|       0.674497472|       9589766.961|
+-------+------------------+---------+--------+------------------+------------------+



In [15]:
# stats for specific columns
csvNewDataFrame.describe("gains_value").show()

+-------+------------------+
|summary|       gains_value|
+-------+------------------+
|  count|                10|
|   mean|     -0.4855726898|
| stddev|0.6677054107117851|
|    min|       -1.58036926|
|    max|       0.674497472|
+-------+------------------+



In [16]:
# collect() method returns a list
meanCollectionList = csvNewDataFrame.describe("gains_value").collect()
print(meanCollectionList[1][0],'=',meanCollectionList[1][1])

mean = -0.4855726898


# <span style="color:blue">Cleaning Data</span>
## *Finding number of missing value in each column*


In [17]:
# finding number of missing value in each column
from pyspark.sql.functions import col
totalNumberOfRow = csvNewDataFrame.count()
print('Number of rows in csvNewDataFrame = {0}'.format(totalNumberOfRow))
collectNumRowsInEachCol = csvNewDataFrame.describe().filter(col("summary") == "count").collect()
print("Number of rows in each column:",collectNumRowsInEachCol)

Number of rows in csvNewDataFrame = 11
Number of rows in each column: [Row(summary='count', client_id='11', date='11', weekdays='9', gains_value='10', prices='10')]


# *Handling missing values*
<span style="color:blue"><b>There are many ways to handle missing values in columns. Some of them are discussed here</b></span>
## Deleting rows containig null value

In [18]:
csvNewDataFrame.na.drop().show() # it deletes all rows which have atleast one null value
# actual dataframe is not changed because we didnt assign

+---------+---------+--------+------------+-----------+-----+
|client_id|     date|weekdays| gains_value|     prices|   up|
+---------+---------+--------+------------+-----------+-----+
|      0.0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
|      2.0| 5/2/2008|     Fri|-0.868509701|9589766.961|false|
|      5.0| 5/5/2008|     Mon|-0.681516369|949579.8802|false|
|      7.0| 5/7/2008|     Wed| 0.674497472|7517014.783| true|
|      8.0| 5/8/2008|     Thu|-1.184100866|1920959.542|false|
|      0.0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
+---------+---------+--------+------------+-----------+-----+



In [19]:
csvNewDataFrame.show() # actual dataset unchanged

+---------+---------+--------+------------+-----------+-----+
|client_id|     date|weekdays| gains_value|     prices|   up|
+---------+---------+--------+------------+-----------+-----+
|      0.0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
|      1.0| 5/1/2008|     Thu|        null|3167180.737| true|
|      2.0| 5/2/2008|     Fri|-0.868509701|9589766.961|false|
|      3.0| 5/3/2008|     Sat|-0.427010839|       null|false|
|      4.0| 5/4/2008|    null| 0.253255365|937163.4438| true|
|      5.0| 5/5/2008|     Mon|-0.681516369|949579.8802|false|
|      6.0| 5/6/2008|     Tue| 0.007191158|7268426.907| null|
|      7.0| 5/7/2008|     Wed| 0.674497472|7517014.783| true|
|      8.0| 5/8/2008|     Thu|-1.184100866|1920959.542|false|
|      9.0| 5/9/2008|    null| -1.58036926| 8456240.62| null|
|      0.0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
+---------+---------+--------+------------+-----------+-----+



In [20]:
csvNewDataFrame.na.drop(subset = ['weekdays']).show() # delete null using specific column

+---------+---------+--------+------------+-----------+-----+
|client_id|     date|weekdays| gains_value|     prices|   up|
+---------+---------+--------+------------+-----------+-----+
|      0.0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
|      1.0| 5/1/2008|     Thu|        null|3167180.737| true|
|      2.0| 5/2/2008|     Fri|-0.868509701|9589766.961|false|
|      3.0| 5/3/2008|     Sat|-0.427010839|       null|false|
|      5.0| 5/5/2008|     Mon|-0.681516369|949579.8802|false|
|      6.0| 5/6/2008|     Tue| 0.007191158|7268426.907| null|
|      7.0| 5/7/2008|     Wed| 0.674497472|7517014.783| true|
|      8.0| 5/8/2008|     Thu|-1.184100866|1920959.542|false|
|      0.0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
+---------+---------+--------+------------+-----------+-----+



In [21]:
csvNewDataFrame.na.drop(subset = ['weekdays','prices']).show() # delete null using specific list of columns

+---------+---------+--------+------------+-----------+-----+
|client_id|     date|weekdays| gains_value|     prices|   up|
+---------+---------+--------+------------+-----------+-----+
|      0.0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
|      1.0| 5/1/2008|     Thu|        null|3167180.737| true|
|      2.0| 5/2/2008|     Fri|-0.868509701|9589766.961|false|
|      5.0| 5/5/2008|     Mon|-0.681516369|949579.8802|false|
|      6.0| 5/6/2008|     Tue| 0.007191158|7268426.907| null|
|      7.0| 5/7/2008|     Wed| 0.674497472|7517014.783| true|
|      8.0| 5/8/2008|     Thu|-1.184100866|1920959.542|false|
|      0.0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
+---------+---------+--------+------------+-----------+-----+



## Filling missing data

In [22]:
csvNewDataFrame.na.fill(0).show() # it fills only numeric missing data with value 0

+---------+---------+--------+------------+-----------+-----+
|client_id|     date|weekdays| gains_value|     prices|   up|
+---------+---------+--------+------------+-----------+-----+
|      0.0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
|      1.0| 5/1/2008|     Thu|         0.0|3167180.737| true|
|      2.0| 5/2/2008|     Fri|-0.868509701|9589766.961|false|
|      3.0| 5/3/2008|     Sat|-0.427010839|        0.0|false|
|      4.0| 5/4/2008|    null| 0.253255365|937163.4438| true|
|      5.0| 5/5/2008|     Mon|-0.681516369|949579.8802|false|
|      6.0| 5/6/2008|     Tue| 0.007191158|7268426.907| null|
|      7.0| 5/7/2008|     Wed| 0.674497472|7517014.783| true|
|      8.0| 5/8/2008|     Thu|-1.184100866|1920959.542|false|
|      9.0| 5/9/2008|    null| -1.58036926| 8456240.62| null|
|      0.0|4/30/2008|     Wed|-0.524581929|7791404.009|false|
+---------+---------+--------+------------+-----------+-----+



In [23]:
csvNewDataFrame.na.fill('weekdays missing',subset=['weekdays']).show() 
# it fills  "weekdays" missing data only with the value "weekdays missing"

+---------+---------+----------------+------------+-----------+-----+
|client_id|     date|        weekdays| gains_value|     prices|   up|
+---------+---------+----------------+------------+-----------+-----+
|      0.0|4/30/2008|             Wed|-0.524581929|7791404.009|false|
|      1.0| 5/1/2008|             Thu|        null|3167180.737| true|
|      2.0| 5/2/2008|             Fri|-0.868509701|9589766.961|false|
|      3.0| 5/3/2008|             Sat|-0.427010839|       null|false|
|      4.0| 5/4/2008|weekdays missing| 0.253255365|937163.4438| true|
|      5.0| 5/5/2008|             Mon|-0.681516369|949579.8802|false|
|      6.0| 5/6/2008|             Tue| 0.007191158|7268426.907| null|
|      7.0| 5/7/2008|             Wed| 0.674497472|7517014.783| true|
|      8.0| 5/8/2008|             Thu|-1.184100866|1920959.542|false|
|      9.0| 5/9/2008|weekdays missing| -1.58036926| 8456240.62| null|
|      0.0|4/30/2008|             Wed|-0.524581929|7791404.009|false|
+---------+---------

## <span style="color:black">*We can use statistical info to fill the missing numeric values*</span>
<span style="color:blue"><b>Let, we want to use 'mean' of 'prices' column and 'standerd deviation' of 'gain_value' column to fill missing values there </b></span>

## <span style="color:blue">*Function for mean and standerd dev*<span>

In [24]:
def getMean(dataframe, colName):
    from pyspark.sql.functions import mean
    meanVal = dataframe.select(mean(dataframe[colName])).collect()
    meanVal = meanVal[0][0]
    return meanVal

def getStddev(dataframe, colName):
    from pyspark.sql.functions import stddev
    stddevVal = dataframe.select(stddev(dataframe[colName])).collect()
    stddevVal = stddevVal[0][0]
    return stddevVal

In [25]:
meanPrice = getMean(dataframe = csvNewDataFrame, colName = 'prices')
stddevGain = getStddev(dataframe = csvNewDataFrame, colName = 'gains_value')
print('meanPrice = ',meanPrice)
print('stddevGain = ',stddevGain)


meanPrice =  5538914.089199999
stddevGain =  0.6677054107117851


In [26]:
csvNewDataFrame = csvNewDataFrame.na.fill(meanPrice, ['prices'])
csvNewDataFrame = csvNewDataFrame.na.fill(stddevGain, ['gains_value'])
csvNewDataFrame.show()

+---------+---------+--------+------------------+-----------------+-----+
|client_id|     date|weekdays|       gains_value|           prices|   up|
+---------+---------+--------+------------------+-----------------+-----+
|      0.0|4/30/2008|     Wed|      -0.524581929|      7791404.009|false|
|      1.0| 5/1/2008|     Thu|0.6677054107117851|      3167180.737| true|
|      2.0| 5/2/2008|     Fri|      -0.868509701|      9589766.961|false|
|      3.0| 5/3/2008|     Sat|      -0.427010839|5538914.089199999|false|
|      4.0| 5/4/2008|    null|       0.253255365|      937163.4438| true|
|      5.0| 5/5/2008|     Mon|      -0.681516369|      949579.8802|false|
|      6.0| 5/6/2008|     Tue|       0.007191158|      7268426.907| null|
|      7.0| 5/7/2008|     Wed|       0.674497472|      7517014.783| true|
|      8.0| 5/8/2008|     Thu|      -1.184100866|      1920959.542|false|
|      9.0| 5/9/2008|    null|       -1.58036926|       8456240.62| null|
|      0.0|4/30/2008|     Wed|      -0

# <span style="color:blue">User defined function example</span>

In [27]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y').weekday(), IntegerType())
csvNewDataFrame = csvNewDataFrame.withColumn('day_num_of_week', func(col('date')))
csvNewDataFrame.show()
csvNewDataFrame.printSchema()

+---------+---------+--------+------------------+-----------------+-----+---------------+
|client_id|     date|weekdays|       gains_value|           prices|   up|day_num_of_week|
+---------+---------+--------+------------------+-----------------+-----+---------------+
|      0.0|4/30/2008|     Wed|      -0.524581929|      7791404.009|false|              2|
|      1.0| 5/1/2008|     Thu|0.6677054107117851|      3167180.737| true|              3|
|      2.0| 5/2/2008|     Fri|      -0.868509701|      9589766.961|false|              4|
|      3.0| 5/3/2008|     Sat|      -0.427010839|5538914.089199999|false|              5|
|      4.0| 5/4/2008|    null|       0.253255365|      937163.4438| true|              6|
|      5.0| 5/5/2008|     Mon|      -0.681516369|      949579.8802|false|              0|
|      6.0| 5/6/2008|     Tue|       0.007191158|      7268426.907| null|              1|
|      7.0| 5/7/2008|     Wed|       0.674497472|      7517014.783| true|              2|
|      8.0

In [29]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
def findMultiplication(gain, price):
    ratio = float(price)*float(gain)
    return ratio
    
userDefiendFuncForMultiplication =  udf(findMultiplication, DoubleType())
csvNewDataFrame = csvNewDataFrame.withColumn('priceMulGain', userDefiendFuncForMultiplication(col('gains_value'),col('prices')))
csvNewDataFrame.show()


+---------+---------+--------+------------------+-----------------+-----+---------------+--------------------+
|client_id|     date|weekdays|       gains_value|           prices|   up|day_num_of_week|        priceMulGain|
+---------+---------+--------+------------------+-----------------+-----+---------------+--------------------+
|      0.0|4/30/2008|     Wed|      -0.524581929|      7791404.009|false|              2| -4087229.7446595533|
|      1.0| 5/1/2008|     Thu|0.6677054107117851|      3167180.737| true|              3|  2114743.7147970395|
|      2.0| 5/2/2008|     Fri|      -0.868509701|      9589766.961|false|              4|  -8328805.635957788|
|      3.0| 5/3/2008|     Sat|      -0.427010839|5538914.089199999|false|              5| -2365176.3523782124|
|      4.0| 5/4/2008|    null|       0.253255365|      937163.4438| true|              6|    237341.670024226|
|      5.0| 5/5/2008|     Mon|      -0.681516369|      949579.8802|false|              0|   -647154.232029359|
|

## Delete duplicate rows
1st and last row is exactly same, so one of them shuld be removed

In [32]:
csvNewDataFrame = csvNewDataFrame.dropDuplicates()
csvNewDataFrame.show()

+---------+---------+--------+------------------+-----------------+-----+---------------+--------------------+
|client_id|     date|weekdays|       gains_value|           prices|   up|day_num_of_week|        priceMulGain|
+---------+---------+--------+------------------+-----------------+-----+---------------+--------------------+
|      4.0| 5/4/2008|    null|       0.253255365|      937163.4438| true|              6|    237341.670024226|
|      5.0| 5/5/2008|     Mon|      -0.681516369|      949579.8802|false|              0|   -647154.232029359|
|      9.0| 5/9/2008|    null|       -1.58036926|       8456240.62| null|              4|-1.336398273101134E7|
|      8.0| 5/8/2008|     Thu|      -1.184100866|      1920959.542|false|              3| -2274609.8572331634|
|      3.0| 5/3/2008|     Sat|      -0.427010839|5538914.089199999|false|              5| -2365176.3523782124|
|      6.0| 5/6/2008|     Tue|       0.007191158|      7268426.907| null|              1|    52268.4062996883|
|

## Finding unique values from a column

In [36]:
print(csvNewDataFrame.select('up').distinct().count())
csvNewDataFrame.select('up').distinct().show()

3
+-----+
|   up|
+-----+
| null|
| true|
|false|
+-----+



## Spark SQL example

In [43]:
csvNewDataFrame.groupBy(["day_num_of_week"]).count().orderBy('day_num_of_week').show()

+---------------+-----+
|day_num_of_week|count|
+---------------+-----+
|              0|    1|
|              1|    1|
|              2|    2|
|              3|    2|
|              4|    2|
|              5|    1|
|              6|    1|
+---------------+-----+



In [49]:
csvNewDataFrame.groupBy("day_num_of_week").max("prices","gains_value").orderBy('day_num_of_week').show()

+---------------+-----------------+------------------+
|day_num_of_week|      max(prices)|  max(gains_value)|
+---------------+-----------------+------------------+
|              0|      949579.8802|      -0.681516369|
|              1|      7268426.907|       0.007191158|
|              2|      7791404.009|       0.674497472|
|              3|      3167180.737|0.6677054107117851|
|              4|      9589766.961|      -0.868509701|
|              5|5538914.089199999|      -0.427010839|
|              6|      937163.4438|       0.253255365|
+---------------+-----------------+------------------+



In [50]:
csvNewDataFrame.groupBy("day_num_of_week").agg({'prices':'avg','gains_value':'max'}).orderBy('day_num_of_week').show()

+---------------+------------------+-----------------+
|day_num_of_week|  max(gains_value)|      avg(prices)|
+---------------+------------------+-----------------+
|              0|      -0.681516369|      949579.8802|
|              1|       0.007191158|      7268426.907|
|              2|       0.674497472|      7654209.396|
|              3|0.6677054107117851|     2544070.1395|
|              4|      -0.868509701|     9023003.7905|
|              5|      -0.427010839|5538914.089199999|
|              6|       0.253255365|      937163.4438|
+---------------+------------------+-----------------+



In [54]:
csvNewDataFrame.groupBy("day_num_of_week").min("prices","gains_value").orderBy('day_num_of_week',ascending=False).show()

+---------------+-----------------+----------------+
|day_num_of_week|      min(prices)|min(gains_value)|
+---------------+-----------------+----------------+
|              6|      937163.4438|     0.253255365|
|              5|5538914.089199999|    -0.427010839|
|              4|       8456240.62|     -1.58036926|
|              3|      1920959.542|    -1.184100866|
|              2|      7517014.783|    -0.524581929|
|              1|      7268426.907|     0.007191158|
|              0|      949579.8802|    -0.681516369|
+---------------+-----------------+----------------+



In [59]:
csvNewDataFrame.filter(csvNewDataFrame['weekdays']!='null').filter(csvNewDataFrame['weekdays']!='Mon').show()

+---------+---------+--------+------------------+-----------------+-----+---------------+-------------------+
|client_id|     date|weekdays|       gains_value|           prices|   up|day_num_of_week|       priceMulGain|
+---------+---------+--------+------------------+-----------------+-----+---------------+-------------------+
|      8.0| 5/8/2008|     Thu|      -1.184100866|      1920959.542|false|              3|-2274609.8572331634|
|      3.0| 5/3/2008|     Sat|      -0.427010839|5538914.089199999|false|              5|-2365176.3523782124|
|      6.0| 5/6/2008|     Tue|       0.007191158|      7268426.907| null|              1|   52268.4062996883|
|      1.0| 5/1/2008|     Thu|0.6677054107117851|      3167180.737| true|              3| 2114743.7147970395|
|      2.0| 5/2/2008|     Fri|      -0.868509701|      9589766.961|false|              4| -8328805.635957788|
|      7.0| 5/7/2008|     Wed|       0.674497472|      7517014.783| true|              2|  5070207.468120128|
|      0.0

In [61]:
from pyspark.sql.functions import countDistinct, avg, stddev,sum
csvNewDataFrame.select(stddev('prices')).show()

+-------------------+
|stddev_samp(prices)|
+-------------------+
| 3288759.5002916446|
+-------------------+



In [63]:
csvNewDataFrame.groupBy("day_num_of_week").agg(avg("prices").alias("avgPrice")).orderBy("avgPrice",ascending=False).show()

+---------------+-----------------+
|day_num_of_week|         avgPrice|
+---------------+-----------------+
|              4|     9023003.7905|
|              2|      7654209.396|
|              1|      7268426.907|
|              5|5538914.089199999|
|              3|     2544070.1395|
|              0|      949579.8802|
|              6|      937163.4438|
+---------------+-----------------+

